In [2]:
# Important libaries
import pandas as pd
import nltk

In [3]:
df = pd.read_table("/kaggle/input/LIARbyYang/train.tsv", header = None)
df.columns = ["ID", "label", "statement", "subject", "speaker", "speaker_job", "state", "party", "pof_count", "false_count", "barelytrue_count", "halftrue_count", "mostlytrue_count", "context"]

In [4]:
df.head(10)


,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Merge label

In [5]:
merger = { 'pants-fire' : 0,
           'false' : 0,
           'barely-true': 0,
           'half-true': 0,
           'mostly-true': 1,
           'true': 1}
df["label"].replace(merger, inplace=True)
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,0,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,0,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Clean data

In [12]:
df['context'].replace('\.','',regex=True,inplace=True) # remove period
df.loc[df.context.str.contains('mailer', case = False, na = False),'context'] = 'a mailer' # replace "$mailer$" with just "a mailer"
df.loc[df.context.str.contains('floor speech', case = False, na = False),'context'] = 'a floor speech' # merge floor speeches
df.loc[df.context.str.contains('a speech', case = False, na = False),'context'] = 'a speech' # merge speeches
df.loc[df.context.str.contains('Denver', na = False, case = False),'context'] = 'Denver' # merge speeches
df.loc[df.context.str.contains('a website', case = False, na = False),'context'] = 'a website' # merge a website
df.loc[df.context.str.contains('online video', case = False, na = False),'context'] = 'an online video' # merge an online video
df.loc[df.context.str.contains('a Democratic debate', case = False, na = False),'context'] = 'a Democratic debate' # merge a Democratic debate
df.loc[(df['context'].str.contains('an interview', case = False, na=False)) & (df['context'].str.contains('news', case = False, na=False)),'context'] = 'an interview with news' # merge a Democratic debate an interview & news
df.loc[(df['context'].str.contains('an interview on', case = False, na=False)),'context'] = 'an interview with news'  # merge an interview on change to an interview with news

In [7]:
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech
2,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview with news
5,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,0,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release
7,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a Democratic debate
8,5602.json,0,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Test

Show unique value for "context"

In [8]:
df["context"].unique()

array(['a mailer', 'a floor speech', 'Denver', ...,
       'a recorded telephone message to Cranston residents',
       'interview on "The Colbert Report"',
       "a televised debate on Miami's WPLG-10 against Kendrick Meek"],
      dtype=object)

Show all rows that contain:

In [9]:
df[df['context'].str.contains('speech', na=False)]

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech
10,7115.json,1,"For the first time in history, the share of th...",elections,robert-menendez,U.S. Senator,New Jersey,democrat,1.0,3.0,1.0,3.0,0.0,a speech
26,12517.json,1,Hate crimes against American Muslims and mosqu...,"crime,diversity,homeland-security,terrorism",hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,69.0,76.0,7.0,a speech
33,8239.json,1,We cut business taxes so today 70 percent of o...,"small-business,taxes",rick-scott,Governor,Florida,republican,28.0,23.0,38.0,34.0,7.0,a speech
35,7550.json,1,We have a federal government that thinks they ...,"consumer-safety,corrections-and-updates,govern...",ted-cruz,Senator,Texas,republican,36.0,33.0,15.0,19.0,8.0,a speech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10183,1468.json,0,"Vice President Joe Biden's meeting ""with the t...","economy,stimulus,transparency",sarah-palin,NaN,Alaska,republican,9.0,19.0,9.0,6.0,6.0,a speech
10190,1991.json,0,"Says his GOP challenger, Doug MacGinnitie, ""ha...",voting-record,brian-kemp,Secretary of State,Georgia,republican,0.0,2.0,0.0,0.0,0.0,a Cobb County Republican Party breakfast speec...
10207,6178.json,0,Obamacare means that for up to 20 million Amer...,health-care,mitt-romney,Former governor,Massachusetts,republican,34.0,32.0,58.0,33.0,19.0,a speech
10211,1550.json,1,Debt has almost doubled in Austin under Gov. P...,economy,bill-white,Former mayor of Houston,Texas,democrat,2.0,3.0,5.0,7.0,3.0,a speech


In [10]:
#E
#filtered_df=df[df['context'].str.contains("an interview on" , case = False, na=False)]
filtered_df = df[(df['context'].str.contains('an interview', case = False, na=False)) & (df['context'].str.contains('news', case = False, na=False))]
filtered_df[['label', 'context']]

,label,context
4,0,an interview with news
12,0,an interview with news
19,1,an interview with news
47,0,an interview with news
48,0,an interview with news
...,...,...
10191,1,an interview with news
10213,0,an interview with news
10226,1,an interview with news
10230,0,an interview with news
